In [11]:
#%% baseline model
from keras.layers import Conv2D
from keras.layers import UpSampling2D
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import RepeatVector
from keras.layers import Reshape
from keras.layers.merge import concatenate
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
#from keras.utils import plot_model
from os import listdir
from pickle import load
import numpy as np

In [2]:
def define_model_baseline(img_h, img_w):

    # encoder
    inputs = Input(shape=(img_h, img_w, 3,))
    encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(inputs)
    encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
    encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
    encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
    encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
    encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
    encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
    encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
    
    # decoder
    
    decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)
    decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)
    decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
    decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
    decoder_output = Conv2D(3, (3, 3), activation='tanh', padding='same')(decoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)


    # tie it together 
    model = Model(inputs=inputs, outputs=decoder_output)
    model.compile(loss='mse', optimizer='adam', metrics=['acc'])
    # summarize model
    print(model.summary())
    #plot_model(model, to_file='autoencoder_colorization_baseline.png', show_shapes=True)
    
    return model

In [3]:
model=define_model_baseline(256, 256)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 256)       295168    
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 256)       590080    
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 512)       118016

In [ ]:
!unzip boh.zip 

In [7]:
training_dir = '../content/boh/'

Orig = []
BW = []
for filename in listdir(training_dir):
  data_orig = img_to_array(load_img(training_dir+filename, target_size=(256,256)))
  data_orig = 1.0/255*data_orig
  data_BW = img_to_array(load_img(training_dir+filename, target_size=(256,256), color_mode='grayscale'))
  data_BW = np.reshape(data_BW, newshape=(256,256))
  data_BW_3 = np.zeros((256, 256, 3))
  data_BW_3[:,:,0] = data_BW 
  data_BW_3[:,:,1] = data_BW
  data_BW_3[:,:,2] = data_BW
  data_BW_3 = 1.0/255*data_BW_3
  Orig.append(data_orig)
  BW.append(data_BW_3)
Orig = np.array(Orig)
BW = np.array(BW)

In [12]:
train_orig, test_orig, train_BW, test_BW = train_test_split(Orig, BW, test_size=0.2, random_state=42)

In [15]:
fit_history = model.fit(np.array(BW), np.array(Orig), epochs=3, verbose=1)

Epoch 1/3


KeyboardInterrupt: ignored

In [ ]:
model.save('model.h5')

fittare su train

testare su tutto dataset

salvare immagini

calcolare metriche 

immagini filtered